In [1]:
#Socket client example in python
 
import socket   #for sockets
import sys  #for exit
import struct
from __future__ import division
import time
import sys,io
import string

In [7]:
s.close()

In [4]:
 
#create an INET, STREAMing socket
try:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
except socket.error:
    print 'Failed to create socket'
    sys.exit()
     
print 'Socket Created'
 
host = '1.2.3.4';
port = 2000;
 
try:
    remote_ip = socket.gethostbyname( host )
 
except socket.gaierror:
    #could not resolve
    print 'Hostname could not be resolved. Exiting'
    sys.exit()
 
#Connect to remote server
s.connect((host , port))
 
print 'Socket Connected to ' + host + ' on ip ' + remote_ip

Socket Created
Socket Connected to 1.2.3.4 on ip 1.2.3.4


In [5]:
targets={'ANY':'\x00',
         'MB' :'\x01',
         'HC' :'\x04',
         'HC+':'\x0d',
         'AZM':'\x10',
         'ALT':'\x11',
         'NET':'\x20',
         'GPS':'\xb0',
        }
commands={
          'MC_GET_POSITION':'\x01',
          'MC_GOTO_FAST':'\x02',
          'MC_SET_POSITION':'\x04',
          'MC_SET_POS_GUIDERATE':'\x06',
          'MC_SET_NEG_GUIDERATE':'\x07',
          'MC_LEVEL_START':'\x0b',
          'MC_GOTO_SLOW':'\x17',
          'MC_SEEK_INDEX':'\x19',
          'GET_VER':'\xfe',
         }

trgid={}
for k in targets.keys():
    trgid[ord(targets[k])]=k
cmdid={}
for k in commands.keys():
    cmdid[ord(commands[k])]=k
    
def checksum(msg):
    return ((~sum([ord(c) for c in msg]) + 1) ) & 0xFF

def f2dms(f):
    d=360*f
    dd=int(d)
    mm=int((d-dd)*60)
    ss=(d-dd-mm/60)*3600
    return dd,mm,ss

def parse_pos(d):
    if len(d)==3 :
        pos=struct.unpack('!i','\x00'+d)[0]/2**24
        return u'%03d°%02d\'%04.1f"' % f2dms(pos)
    else :
        return ''

parsers={
    '\x01':parse_pos,
    '\x02':parse_pos,
    '\x04':parse_pos,
    '\x06':parse_pos,
    '\x07':parse_pos,
    '\x17':parse_pos,
}

def send_cmd(sck, src='NET', trg='AZM', cmd='GET_VER', data=''):
    pre='\x3b'
    msg=targets[src]+targets[trg]+commands[cmd]+data
    msg=chr(len(msg))+msg
    sck.sendall(pre+msg+chr(checksum(msg)))

def dprint(m):
    for c in m:
        if ord(c)==0x3b :
            print
        print "0x%02x" %ord(c),
    print

def split_msgs(r,debug=False):
    l=r.find('\x3b')+1
    p=l
    ml=[]
    while p>-1:
        p=r.find('\x3b',l)
        #if debug : print l, p
        ml.append(r[l:p]+r[p])
        l=p+1
    if debug : print ml
    return ml
    
def parse_msg(m, debug=False):
    '''
    Parse bytes byond 0x3b. 
    Do not pass the message with preamble!
    '''
    l=ord(m[0])+1
    msg=m[:l]
    if debug :
        print 'Parse:',
        dprint(msg)
    if  chr(checksum(msg)) != m[l] :
        print 'Checksum error: %x vs. %02x' % (checksum(msg) , ord(m[l]))
        dprint(m)
    l,src,dst,mid=struct.unpack('4B',msg[:4])
    dat=msg[4:l+1]
    #print 'len:', l
    return l, src, dst, mid, dat


def print_msg(l,s,d,i,dat):
    try :
        print '[%d] %4s => %4s (%12s): ' % (l,trgid[s],trgid[d],cmdid[i]),
    except KeyError :
        print '[%d] %02x => %02x (%02x): ' % (l,s,d,i),
    if chr(i) in parsers:
        print parsers[chr(i)](dat),
    else :
        dprint(dat)

def rcv_msg(sck, trg='NET', debug=False):
    r = sck.recv(4096)
    #dprint(r)
    we=ord(targets[trg])
    ml=[]
    for m in split_msgs(r,debug=debug):
        if debug :
            print 'RCV:',
            dprint(m)
        try :
            l,s,d,i,dat=parse_msg(m,debug=debug)
            if trg!='ANY' and d!=we and s!=we:
                print 'Not for us (%02x): %02x ->  %02x' % (we, s, d)
                continue
            if debug : print_msg(l,s,d,i,dat)
            ml.append([l,s,d,i,dat])
        except IndexError :
            print 'Parse ERROR:',
            dprint(m)
    return ml


def get_pos(sck,trg='ALT'):
    send_cmd(s,trg=trg,cmd='MC_GET_POSITION')
    time.sleep(0.05)
    ml=rcv_msg(s)
    dat='\x00'+ml[-1][-1]
    return struct.unpack('!i',dat)[0]/2**24
    
def get_ver(sck,trg='MB'):
    send_cmd(sck,trg=trg,cmd='GET_VER')
    msg=rcv_msg(sck)[-1]
    return tuple(['%02x' % ord(c) for c in msg[-1]])

def show_pos(sck):
    azm=get_pos(sck,trg='AZM')
    print u'\rAZM %03d°%02d\'%04.1f"' % f2dms(azm),
    time.sleep(0.1)
    alt=get_pos(sck,trg='ALT')
    print u'ALT %+02d°%02d\'%04.1f"' % f2dms(alt),


In [7]:
%%writefile nexstarevo.py
#!env python

# NexStar Evolution communication library
# (L) by Paweł T. Jochym <jochym@gmail.com>
# This code is under GPL 3.0 license


# ID tables
targets={'ANY':'\x00',
         'MB' :'\x01',
         'HC' :'\x04',
         'HC+':'\x0d',
         'AZM':'\x10',
         'ALT':'\x11',
         'NET':'\x20',
         'GPS':'\xb0',
        }
commands={
          'MC_GET_POSITION':'\x01',
          'MC_GOTO_FAST':'\x02',
          'MC_SET_POSITION':'\x04',
          'MC_SET_POS_GUIDERATE':'\x06',
          'MC_SET_NEG_GUIDERATE':'\x07',
          'MC_LEVEL_START':'\x0b',
          'MC_GOTO_SLOW':'\x17',
          'MC_SEEK_INDEX':'\x19',
          'GET_VER':'\xfe',
         }

trgid={}
for k in targets.keys():
    trgid[ord(targets[k])]=k
cmdid={}
for k in commands.keys():
    cmdid[ord(commands[k])]=k

# Utility functions
def checksum(self, msg):
    return ((~sum([ord(c) for c in msg]) + 1) ) & 0xFF

def dprint(m):
    for c in m:
        if ord(c)==0x3b :
            print
        print "0x%02x" %ord(c),
    print


def split_msgs(r,debug=False):
    l=r.find('\x3b')+1
    p=l
    ml=[]
    while p>-1:
        p=r.find('\x3b',l)
        #if debug : print l, p
        ml.append(r[l:p]+r[p])
        l=p+1
    if debug : print ml
    return ml

def f2dms(f):
    '''
    Convert fraction of the full rotation to DMS triple (degrees).
    '''
    d=360*f
    dd=int(d)
    mm=int((d-dd)*60)
    ss=(d-dd-mm/60)*3600
    return dd,mm,ss

def parse_pos(d):
    if len(d)==3 :
        pos=struct.unpack('!i','\x00'+d)[0]/2**24
        return u'%03d°%02d\'%04.1f"' % f2dms(pos)
    else :
        return ''


# Message classes
class nse_msg:
    
    def __init__(self, data=None):
        if data is not None :
            self.l, self.src, self.dst, self.mid, self.dat=self.parse(data)
        else :
            pass
        
    def parse(self, m, debug=False):
        '''
        Parse bytes byond 0x3b. The data should be a byte string.
        Do not pass the message with preamble! 
        The function assumes the first byte is a length byte.
        The structure of the message (bytes):
            <preamble=0x3b> -- should be removed
            <packet len> -- assumed to be the first byte here
            <source device #>
            <destination device #>
            <message ID>
            <message data bytes>
            <checksum byte>
        '''
        l=ord(m[0])+1gather
        msg=m[:l]
        if debug :
            print 'Parse:',
            dprint(msg)
        if  chr(checksum(msg)) != m[l] :
            print 'Checksum error: %02x vs. %02x' % (checksum(msg) , ord(m[l]))
            dprint(m)
        l,src,dst,mid=struct.unpack('4B',msg[:4])
        dat=msg[4:l+1]
        #print 'len:', l
        return l, src, dst, mid, dat
    

class nse_pos_msg(nse_msg):
    
    nse_msg.dt['\x01']
parsers={
    '\x01':parse_pos,
    '\x02':parse_pos,
    '\x04':parse_pos,
    '\x06':parse_pos,
    '\x07':parse_pos,
    '\x17':parse_pos,
}
    
    def __init__(self, data=None):
        nse_msg.__init__(self,data)

Overwriting nexstarevo.py


In [ ]:
while True :
    for m in rcv_msg(s,'ANY'):
        print_msg(*m)
        print
    sys.stdout.flush()
    time.sleep(0.05)

In [15]:
send_cmd(s,src='NET',trg='AZM',cmd='GET_VER')
print rcv_msg(s,trg='NET')
time.sleep(0.1)
send_cmd(s,src='NET',trg='ALT',cmd='GET_VER')
print rcv_msg(s,trg='NET')

[[3, 32, 16, 254, ''], [7, 16, 32, 254, '\x07\n\x10\r']]
[[3, 32, 17, 254, ''], [7, 17, 32, 254, '\x07\n\x10\r']]


In [24]:
send_cmd(s,src='NET',trg='HC+',cmd='GET_VER')
rcv_msg(s,trg='ANY')

[[3, 32, 13, 254, '']]

In [277]:
send_cmd(s,trg='AZM',cmd='MC_GET_POSITION')
rcv_msg(s)
send_cmd(s,trg='ALT',cmd='MC_GET_POSITION')
rcv_msg(s)

[[3, 32, 17, 1, ''], [6, 17, 32, 1, '\x00\x00\x00']]

In [287]:
print get_ver(s,'HC')

()


In [279]:
send_cmd(s,trg='HC+',cmd='MC_GET_POSITION')
rcv_msg(s)

[[3, 32, 4, 1, '']]

In [289]:
print get_ver(s,'MB'), get_ver(s,'HC'), get_ver(s,'ALT'), get_ver(s,'AZM')

() () ('07', '0a', '10', '0d') ('07', '0a', '10', '0d')


In [281]:
show_pos(s)
sys.stdout.flush()

AZM 000°00'00.0" ALT +0°00'00.0"


In [284]:
for i in range(10):
    show_pos(s)
    sys.stdout.flush()
    time.sleep(1)

AZM 000°00'00.0" ALT +0°00'00.0"


In [173]:
def goto(s,alt,azm,spd='FAST'):
    al=int(alt*2**24)
    az=int(azm*2**24)
    cmd='MC_GOTO_'+spd
    send_cmd(s,trg='ALT',cmd=cmd,data=struct.pack('!i',al)[1:])
    time.sleep(0.1)
    send_cmd(s,trg='AZM',cmd=cmd,data=struct.pack('!i',az)[1:])

In [241]:
goto(s,0.0,0.0,spd='FAST')

In [199]:
def frac_to_fp24(x):
    return struct.pack('!i',int(abs(x)*2**24))[1:]

def set_guiderate(s,altgr,azmgr):
    ald='POS' if (altgr > 0) else 'NEG'
    azd='POS' if (azmgr > 0) else 'NEG'
    send_cmd(s,trg='ALT',cmd='MC_SET_'+ald+'_GUIDERATE',data=frac_to_fp24(altgr))
    time.sleep(0.05)
    send_cmd(s,trg='AZM',cmd='MC_SET_'+azd+'_GUIDERATE',data=frac_to_fp24(azmgr))    
    time.sleep(0.05)


In [233]:
set_guiderate(s,0,0)

In [240]:
send_cmd(s,trg='ALT',cmd='MC_LEVEL_START')
rcv_msg(s,debug=True)

[3] 20 => 11 (0b): 
[4] 11 => 20 (f0):  0x0b


[[3, 32, 17, 11, ''], [4, 17, 32, 240, '\x0b']]

In [104]:
struct.unpack('!i','\x00\x02\x7d\xc6')[0]/2**24

0.009731650352478027

In [96]:
print((3+30/60+12/3600)/360)
for c in struct.pack('f',(3+30/60+12/3600)/360):
    print('%02x' % ord(c))
print()

0.00973148148148
cb
70
1f
3c



In [87]:
''.split()

[]

In [103]:
a='\x3b1aaaa\x3b2bbbbb\x3b3ccccc'

In [105]:
a.split('\x3b')[1:]

['1aaaa', '2bbbbb', '3ccccc']

In [ ]:
a.rfind()

In [143]:
split_msgs(a)

['1aaaa', '2bbbbb', '3cccc']

In [142]:
a

';1aaaa;2bbbbb;3ccccc'

In [126]:
for c in a:
    print '%02x' % ord(c) ,

3b 31 61 61 61 61 3b 32 62 62 62 62 62 3b 33 63 63 63 63 63
